# Converting a jupyter notebook to W&B report
this is a subtitle

Let's install the required libs:

In [60]:
# this is bash
!pip install wandb execnb ipynbname

We are going to read this notebook and create a report on W&B with it's content

In [42]:
import wandb
import wandb.apis.reports as wb

import ipynbname
from execnb.nbio import read_nb

We can read the notebook by using `read_nb`

In [43]:
fname = "jupyter2report.ipynb"

In [44]:
nb = read_nb(fname)

the output is a dictionary containing the cells:

In [45]:
nb.cells

(#25) [{'cell_type': 'markdown', 'metadata': {}, 'source': '# Converting a jupyter notebook to W&B report\nthis is a subtitle', 'idx_': 0},{'cell_type': 'markdown', 'metadata': {}, 'source': "We are going to read this notebook and create a report on W&B with it's content", 'idx_': 1},{'cell_type': 'code', 'execution_count': 23, 'metadata': {}, 'outputs': [], 'source': 'import wandb\nimport wandb.apis.reports as wb\n\nfrom execnb.nbio import read_nb', 'idx_': 2},{'cell_type': 'markdown', 'metadata': {}, 'source': 'We can read the notebook by using `read_nb`', 'idx_': 3},{'cell_type': 'code', 'execution_count': 24, 'metadata': {}, 'outputs': [], 'source': 'fname = "jupyter2report.ipynb"', 'idx_': 4},{'cell_type': 'code', 'execution_count': 25, 'metadata': {}, 'outputs': [], 'source': 'nb = read_nb(fname)', 'idx_': 5},{'cell_type': 'markdown', 'metadata': {}, 'source': 'the output is a dictionary containing the cells:', 'idx_': 6},{'cell_type': 'code', 'execution_count': 26, 'metadata': {

## Creating a report

In [46]:
api = wandb.Api()

In [47]:
wandb.require('report-editing:v0')

wandb: WARNING This is an experimental feature -- use with caution!


In [48]:
report = api.create_report(project='report-editing')
report.title = 'A report from a Jupyter notebook'
report.description = 'This report comes from this notebook'

let's check the content of a cell:

In [49]:
cell = nb.cells[0]
cell

```json
{ 'cell_type': 'markdown',
  'idx_': 0,
  'metadata': {},
  'source': '# Converting a jupyter notebook to W&B report\nthis is a subtitle'}
```

In [50]:
nb.cells[2]

```json
{ 'cell_type': 'code',
  'execution_count': 23,
  'idx_': 2,
  'metadata': {},
  'outputs': [],
  'source': 'import wandb\n'
            'import wandb.apis.reports as wb\n'
            '\n'
            'from execnb.nbio import read_nb'}
```

In [51]:
nb.cells[-1]

```json
{ 'cell_type': 'code',
  'execution_count': None,
  'idx_': 24,
  'metadata': {},
  'outputs': [],
  'source': ''}
```

In [52]:
def parse_md_cell(cell):
    source = cell.source.splitlines()
    section = []
    for line in source:
        if '### ' in line:
            section += [wb.H3(line.split("### ")[1])]
        elif '## ' in line:
            section += [wb.H2(line.split("## ")[1])]
        elif '# ' in line:
            section += [wb.H1(line.split("# ")[1])]
        else:
            section += [wb.P(line)]
    return section

In [53]:
parse_md_cell(cell)

[H1(text='Converting a jupyter notebook to W&B report'),
 P(text='this is a subtitle')]

In [54]:
def convert_cell(cell):
    "Convert to W&B block"
    if cell.source == '':
        return []
    if cell.cell_type == "markdown":
        return parse_md_cell(cell)
    if cell.cell_type == "code":
        return [wb.CodeBlock(cell.source.splitlines())]

In [55]:
convert_cell(cell)

[H1(text='Converting a jupyter notebook to W&B report'),
 P(text='this is a subtitle')]

In [56]:
blocks = []
for cell in nb.cells:
    blocks += convert_cell(cell)

In [57]:
blocks

[H1(text='Converting a jupyter notebook to W&B report'),
 P(text='this is a subtitle'),
 P(text="We are going to read this notebook and create a report on W&B with it's content"),
 CodeBlock(code=['import wandb', 'import wandb.apis.reports as wb', '', 'from execnb.nbio import read_nb'], language='python'),
 P(text='We can read the notebook by using `read_nb`'),
 CodeBlock(code=['fname = "jupyter2report.ipynb"'], language='python'),
 CodeBlock(code=['nb = read_nb(fname)'], language='python'),
 P(text='the output is a dictionary containing the cells:'),
 CodeBlock(code=['nb.cells'], language='python'),
 H2(text='Creating a report'),
 CodeBlock(code=['api = wandb.Api()'], language='python'),
 CodeBlock(code=["wandb.require('report-editing:v0')"], language='python'),
 CodeBlock(code=["report = api.create_report(project='report-editing')", "report.title = 'A report from a Jupyter notebook'", "report.description = 'This report comes from this notebook'"], language='python'),
 P(text="let's c

In [58]:
report.blocks = blocks

In [59]:
report.save()

Report(project='report-editing', entity='capecape', title='A report from a Jupyter notebook', description='This report comes from this notebook', width='readable', blocks=[H1(text='Converting a jupyter notebook to W&B report'), P(text=['this is a subtitle']), P(text=["We are going to read this notebook and create a report on W&B with it's content"]), CodeBlock(code=['import wandb', 'import wandb.apis.reports as wb', '', 'from execnb.nbio import read_nb'], language='python'), P(text=['We can read the notebook by using `read_nb`']), CodeBlock(code=['fname = "jupyter2report.ipynb"'], language='python'), CodeBlock(code=['nb = read_nb(fname)'], language='python'), P(text=['the output is a dictionary containing the cells:']), CodeBlock(code=['nb.cells'], language='python'), H2(text='Creating a report'), CodeBlock(code=['api = wandb.Api()'], language='python'), CodeBlock(code=["wandb.require('report-editing:v0')"], language='python'), CodeBlock(code=["report = api.create_report(project='report-editing')", "report.title = 'A report from a Jupyter notebook'", "report.description = 'This report comes from this notebook'"], language='python'), P(text=["let's check the content of a cell:"]), CodeBlock(code=['cell = nb.cells[0]', 'cell'], language='python'), CodeBlock(code=['nb.cells[2]'], language='python'), CodeBlock(code=['nb.cells[-1]'], language='python'), CodeBlock(code=['def parse_md_cell(cell):', '    source = cell.source.splitlines()', '    section = []', '    for line in source:', "        if '### ' in line:", '            section += [wb.H3(line.split("### ")[1])]', "        elif '## ' in line:", '            section += [wb.H2(line.split("## ")[1])]', "        elif '# ' in line:", '            section += [wb.H1(line.split("# ")[1])]', '        else:', '            section += [wb.P(line)]', '    return section'], language='python'), CodeBlock(code=['parse_md_cell(cell)'], language='python'), CodeBlock(code=['def convert_cell(cell):', '    "Convert to W&B block"', "    if cell.source == '':", '        return []', '    if cell.cell_type == "markdown":', '        return parse_md_cell(cell)', '    if cell.cell_type == "code":', '        return [wb.CodeBlock(cell.source.splitlines())]'], language='python'), CodeBlock(code=['convert_cell(cell)'], language='python'), CodeBlock(code=['blocks = []', 'for cell in nb.cells:', '    blocks += convert_cell(cell)'], language='python'), CodeBlock(code=['blocks'], language='python'), CodeBlock(code=['report.blocks = blocks'], language='python'), CodeBlock(code=['report.save()'], language='python')])

## Refactor

Let's put everything together, we need 1st to get the nb name automatically

In [76]:
ipynbname.name(), ipynbname.path()

('jupyter2report',
 Path('/Users/tcapelle/wandb/nb_helpers/dev_nbs/jupyter2report.ipynb'))

In [77]:
def notebook2report(project, title, description, fname=None):
    "Converts this notebook into a report"
    # reads notebook
    nb = read_nb(ipynbname.path() if fname is None else fname)
    
    #create report
    api = wandb.Api()
    report = api.create_report(project=project)
    report.title = title
    report.description = description
    
    # parse cells
    blocks = []
    for cell in nb.cells:
        blocks += convert_cell(cell)
    report.blocks = blocks
    return report

In [78]:
project='report-editing'
title = 'A report from a Jupyter notebook'
description = 'This report comes from this notebook'

report = notebook2report(project, title, description)
report.save()

Report(project='report-editing', entity='capecape', title='A report from a Jupyter notebook', description='This report comes from this notebook', width='readable', blocks=[H1(text='Converting a jupyter notebook to W&B report'), P(text=['this is a subtitle']), P(text=["Let's install the required libs:"]), CodeBlock(code=['# this is bash', '!pip install wandb execnb ipynbname'], language='python'), P(text=["We are going to read this notebook and create a report on W&B with it's content"]), CodeBlock(code=['import wandb', 'import wandb.apis.reports as wb', '', 'import ipynbname', 'from execnb.nbio import read_nb'], language='python'), P(text=['We can read the notebook by using `read_nb`']), CodeBlock(code=['fname = "jupyter2report.ipynb"'], language='python'), CodeBlock(code=['nb = read_nb(fname)'], language='python'), P(text=['the output is a dictionary containing the cells:']), CodeBlock(code=['nb.cells'], language='python'), H2(text='Creating a report'), CodeBlock(code=['api = wandb.Api()'], language='python'), CodeBlock(code=["wandb.require('report-editing:v0')"], language='python'), CodeBlock(code=["report = api.create_report(project='report-editing')", "report.title = 'A report from a Jupyter notebook'", "report.description = 'This report comes from this notebook'"], language='python'), P(text=["let's check the content of a cell:"]), CodeBlock(code=['cell = nb.cells[0]', 'cell'], language='python'), CodeBlock(code=['nb.cells[2]'], language='python'), CodeBlock(code=['nb.cells[-1]'], language='python'), CodeBlock(code=['def parse_md_cell(cell):', '    source = cell.source.splitlines()', '    section = []', '    for line in source:', "        if '### ' in line:", '            section += [wb.H3(line.split("### ")[1])]', "        elif '## ' in line:", '            section += [wb.H2(line.split("## ")[1])]', "        elif '# ' in line:", '            section += [wb.H1(line.split("# ")[1])]', '        else:', '            section += [wb.P(line)]', '    return section'], language='python'), CodeBlock(code=['parse_md_cell(cell)'], language='python'), CodeBlock(code=['def convert_cell(cell):', '    "Convert to W&B block"', "    if cell.source == '':", '        return []', '    if cell.cell_type == "markdown":', '        return parse_md_cell(cell)', '    if cell.cell_type == "code":', '        return [wb.CodeBlock(cell.source.splitlines())]'], language='python'), CodeBlock(code=['convert_cell(cell)'], language='python'), CodeBlock(code=['blocks = []', 'for cell in nb.cells:', '    blocks += convert_cell(cell)'], language='python'), CodeBlock(code=['blocks'], language='python'), CodeBlock(code=['report.blocks = blocks'], language='python'), CodeBlock(code=['report.save()'], language='python'), H2(text='Refactor'), P(text=["Let's put everything together, we need 1st to get the nb name automatically"]), CodeBlock(code=[''], language='python'), CodeBlock(code=['nb_path'], language='python'), CodeBlock(code=['def notebook2report(project, title, description, fname=None):', '    "Converts this notebook into a report"', '    # reads notebook', '    nb_fname = ipynbname.name()', '    nb_path = ipynbname.path()', '    nb = read_nb(nb_path if fname is None else fname)', '    ', '    #create report', '    api = wandb.Api()', '    report = api.create_report(project=project)', '    report.title = title', '    report.description = description', '    ', '    # parse cells', '    blocks = []', '    for cell in nb.cells:', '        blocks += convert_cell(cell)', '    report.blocks = blocks', '    return report'], language='python'), CodeBlock(code=["project='report-editing'", "title = 'A report from a Jupyter notebook'", "description = 'This report comes from this notebook'", '', 'report = notebook2report(project, title, description)', 'report.save()'], language='python'), H3(text='Another cool thing we could add is the cell outputs')])

### Another cool thing we could add is the cell outputs